# Gaussian elimination method

by Xiaofeng Liu, Ph.D., P.E.
Associate Professor

Department of Civil and Environmental Engineering

Institute of CyberScience

Penn State University 

223B Sackett Building, University Park, PA 16802

Web: http://water.engr.psu.edu/liu/

---

This notebook is a demonstration of the Gauss elimination method (without and with pivoting) for solving linear equation system.

## Gaussian elimination without pivoting

In [1]:
import numpy as np    

#perfrom Gaussian elimination for the lineary system Ax=b
#on return, the matrix A is destroyed. The solution is
#stored in x. 
def gaussian_elimination(A,b):
    n = len(b)
   
    # forward elimination
    for k in range(0,n-1):
        for i in range(k+1,n):
            if abs(A[k,k]) > 1E-8:
                factor = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - factor*A[k,:]
                b[i] = b[i] - factor*b[k]
            else:
                sys.exit("Singular on diagonal. Pivoting is necessary.")
    
    # back substitution
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = (b[k] - np.dot(A[k,k+1:n],x[k+1:n]))/A[k,k]
    return x

#define the matrix and vector
A = np.array([[2.0,3.0,5.0], [4.0,-5.0,-7.0], [1.0,2.0,-3.0]])
b = np.array([1.0,6.0,2.0])

#call the Gaussian elimination function
x = gaussian_elimination(A,b)

#print out solution
print("solution = ", x)

solution =  [ 1.0942029   0.05072464 -0.26811594]


## Gauss elimination with pivoting

In [5]:
def swap_vector_elements(b,i,j):
    temp = b[i]
    b[i] = b[j]
    b[j] = temp
    
def swap_matrix_elements(A,i,j,n):
    for k in range(0,n):
        temp = A[i,k]
        A[i,k] = A[j,k]
        A[j,k] = temp
    
def gaussian_elimination_pivot(A,b):
    n = len(b)
   
    # forward elimination
    for k in range(0,n-1):
        #Do we need to interchange rows?
        max_val = max(abs(A[k+1:n,k]))        #max value for coeff. down below
        max_ind = np.argmax(abs(A[k+1:n,k]))  #max value's index
        
        if(abs(A[k,k]) < max_val):
            swap_vector_elements(b,k,k+1+max_ind)
            swap_matrix_elements(A,k,k+1+max_ind,n)
           
        for i in range(k+1,n):
            if abs(A[k,k]) > 1E-8:
                factor = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - factor*A[k,:]
                b[i] = b[i] - factor*b[k]
    
    # back substitution
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = (b[k] - np.dot(A[k,k+1:n],x[k+1:n]))/A[k,k]
    return x

A = np.array([[0.0,3,5], [4,-5,-7], [1,2,-3]])
b = np.array([1.0,6,2])

x = gaussian_elimination_pivot(A,b)

#print out solution
print("solution = ", x)

solution =  [1.8875 0.1875 0.0875]


As a check, solving the same linear system with Numpy. The solution should be the same as above. 

In [4]:
import numpy as np

A = np.array([[0.0,3,5], [4,-5,-7], [1,2,-3]])
b = np.array([1.0,6,2])
x = np.linalg.solve(A, b)

print(x)

[1.8875 0.1875 0.0875]


The following code is to test the effect of single, double, and half precision on the results of a system with scaling inbalance. The solution should be **x** = [1.0,1.0]. But depending on what precision is used, the numerical solution may be wrong or the algorithm does not even work. 

In [4]:
import numpy as np    

#perfrom Gaussian elimination for the lineary system Ax=b
#on return, the matrix A is destroyed. The solution is
#stored in x. 
def gaussian_elimination(A,b):
    n = len(b)
   
    # forward elimination
    for k in range(0,n-1):
        for i in range(k+1,n):
            if abs(A[k,k]) > 1E-8:
                factor = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - factor*A[k,:]
                b[i] = b[i] - factor*b[k]
    
    # back substitution
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = (b[k] - np.dot(A[k,k+1:n],x[k+1:n]))/A[k,k]
    return x

#define the matrix and vector
data_type=np.float64   #define the precision, float16 (half), loat32 (single), or, float64 (double)
A = np.array([[2.0,1e8], [1.0,1.0]],dtype=data_type)
b = np.array([1e8,2.0],dtype=data_type)

#call the Gauss elimination function
x = gaussian_elimination(A,b)

#print out solution
print("solution = ", x)

A = np.array([[2.0e-8,1], [1.0,1.0]],dtype=data_type)
b = np.array([1,2.0],dtype=data_type)

#call the Gauss elimination function
x = gaussian_elimination(A,b)

#print out solution
print("solution = ", x)

solution =  [1.00000002 0.99999998]
solution =  [1.00000002 0.99999998]


The following demonstrates the sensitiviy of an ill-conditioned system. Two systems are solved. One is a slight pertubation to the other. The use of half, single, and double precisions affects the solution. The solution should be $x=[1,1]$. However, a slight pertubation may give much different solutions for an ill-conditioned system. 

In [5]:
import numpy as np    

#perfrom Gaussian elimination for the lineary system Ax=b
#on return, the matrix A is destroyed. The solution is
#stored in x. 
def gaussian_elimination(A,b):
    n = len(b)
   
    # Forward elimination
    for k in range(0,n-1):
        for i in range(k+1,n):
            if abs(A[k,k]) > 1E-8:
                factor = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - factor*A[k,:]
                b[i] = b[i] - factor*b[k]
    
    # Back substitution
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = (b[k] - np.dot(A[k,k+1:n],x[k+1:n]))/A[k,k]
    return x

#define the matrix and vector
data_type=np.float16   #or float16, float32, float64

A = np.array([[2.0,1.0], [1.99,0.99]],dtype=data_type)
b = np.array([3,2.98],dtype=data_type)

#call the Gaussian elimination function
x = gaussian_elimination(A,b)

#print out solution
print("solution = ", x)

#add some purturbation to the A matrix
A = np.array([[2.0,1.0], [1.991,0.99]],dtype=data_type)
b = np.array([3,2.98],dtype=data_type)

#call the Gauss elimination function
x = gaussian_elimination(A,b)

#print out solution
print("solution = ", x)

solution =  [1.1 0.8]
solution =  [0.95454545 1.09090909]
